* ****RSNA-MICCAI Brain Tumor Radiogenomic Classification resized Dataset in .npy formate****

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib
import pydicom as dicom
import cv2
import ast
from glob import glob
import warnings
from collections import Counter
import seaborn as sns
from scipy import ndimage, misc
import pydicom
warnings.filterwarnings('ignore')

In [ ]:
path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
train_path = os.path.join(path, 'train')
test_path = os.path.join(path, 'test')


def getFullId(id):
    return str(id).zfill(5)

os.listdir(path)

In [ ]:
train_df = pd.read_csv(path + 'train_labels.csv')
train_df.describe()

In [ ]:
train_df['MGMT_value'].plot.hist()

In [ ]:
train_df.head()

In [ ]:
def resize_volume(img):
    
    desired_depth = 32
    desired_width = 128
    desired_height = 128

    current_depth = img.shape[0]
    current_width = img.shape[1]
    current_height = img.shape[2]
 
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    
    img = ndimage.zoom(img, (depth_factor, width_factor, height_factor), order=1)
    return img


In [ ]:
!mkdir Train_Image_array Test_Image_array 

In [ ]:
import cv2

def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
#     print(data.shape)
    if np.min(data)==np.max(data):
        data = np.zeros((128,128))
        return data
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    
    data = (data * 255).astype(np.uint8)
    data = cv2.resize(data, (128, 128))
    
    return data

def load_dicom_line(path):
    t_paths = sorted(
        glob(os.path.join(path, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    images = []
    for filename in t_paths:
        data = load_dicom(filename)
#         print(data.shape)
        if data.max() == 0:
            data = np.zeros((128,128))
        images.append(data)
#         print(len(images))
    images = np.array(images)
#     print(images.shape)
        
    return images

In [ ]:
%%time
id =0
def get_image(id):
    
    img = []
    _id = getFullId(id)
    for t in ( "T1w", "T1wCE", "FLAIR", "T2w"):
        path = os.path.join(train_path, _id, t)
        frames = np.array(load_dicom_line(path))
#         print(frames.shape)
        img.append(resize_volume(np.array(frames)))
        np.save('./Train_Image_array/'+str(_id)+'.npy', np.array(img).astype(np.uint8))
    return np.array(img)

# img = get_image(0)

In [ ]:
# get_image(train_df['BraTS21ID'][81]).shape

In [ ]:
%%time
from joblib import Parallel, delayed
from tqdm import tqdm
    
results = Parallel(n_jobs = 8, prefer="threads")(delayed(get_image)(id) for id in tqdm(train_df['BraTS21ID']))

In [ ]:
# def get_image(id):
    
#     img = []
#     _id = getFullId(id)
#     for t in ( "T1w", "T1wCE", "FLAIR", "T2w"):
#         path = os.path.join(test_path, _id, t)
#         frames = np.array(load_dicom_line(path))
# #         print(np.array(frames).shape)
#         img.append(resize_volume(np.array(frames)))
#         np.save('Test_Image_array/'+str(_id)+'.npy', np.array(img))
#     return np.array(img)


In [ ]:
# %%time
# from joblib import Parallel, delayed
# from tqdm import tqdm
    
# results = Parallel(n_jobs = 8, prefer="threads")(delayed(get_image)(id) for id in tqdm(os.listdir(test_path)))

In [ ]:
# from matplotlib import animation, rc
# rc('animation', html='jshtml')


# def create_animation(ims):
#     fig = plt.figure(figsize=(6, 6))
#     plt.axis('off')
#     im = plt.imshow(ims[0])

#     def animate_func(i):
#         im.set_array(ims[i])
#         return [im]

#     return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

# create_animation(img[0])

In [ ]:
# from sys import getsizeof
# getsizeof(frames)

In [ ]:
# %%time
# id =0
# def get_image(id):
    
#     img = []
#     _id = getFullId(id)
#     for t in ( "T1w", "T1wCE", "FLAIR", "T2w"):
#         path = os.path.join(train_path, _id, t)
#         frames =[]
#         file_names = sorted(os.listdir(path),
#         key=lambda x: int(x[:-4].split("-")[-1]),)
#         l = len(file_names)
#         if l < 32:
#                 r = range(x)
#         else:
#             d = l // 32
#             r = range(d, 32*d , d)
#         for i in r:
#             img_path = os.path.join(path, file_names[i])
# #             print(img_path)
#             img_2d_scaled = get3ScaledImage(img_path)
# #             print(img_2d_scaled.shape)
#             frames.append(img_2d_scaled)
# #         print(np.array(frames).shape)
# #         img.append(resize_volume(np.array(frames)))
#         img.append(np.array(frames))
# #         np.save('Image_3d_array/'+str(_id)+'.npy', np.array(img))
#     return np.array(img)

# img = get_image(0)

In [ ]:
# def plot_slices(num_rows, num_columns, size, data):

#     fig_width = 13
#     fig_height = (fig_width / num_columns) / (size[1] / size[0]) * num_rows
    
#     fig, ax = plt.subplots(nrows=num_rows, ncols=num_columns, figsize=(fig_width, fig_height))
#     for i in range(num_rows):
#         for j in range(num_columns):
#             ax[i, j].imshow(data[i*num_columns+j])
#             ax[i, j].axis("off")
#     plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
#     plt.show()

# id = 0

# nb_flair = countFlairFiles(id)
# path_flair = getFlairPath(id)
# frames_flair = []

# for i in range(nb_flair):
#     file_name = 'Image-' + str(i+1) + '.dcm'
#     img_path = os.path.join(path_flair, file_name)
#     img_2d_scaled, size = get3ScaledImage(img_path)
#     frames_flair.append(img_2d_scaled)
    
# plot_slices(3, 4, size, frames[90:103])

In [ ]:
# def plot_slices(num_rows, num_columns, size, data):

#     fig_width = 13
#     fig_height = (fig_width / num_columns) / (size[1] / size[0]) * num_rows
    
#     fig, ax = plt.subplots(nrows=num_rows, ncols=num_columns, figsize=(fig_width, fig_height))
#     for i in range(num_rows):
#         for j in range(num_columns):
#             ax[i, j].imshow(data[i*num_columns+j])
#             ax[i, j].axis("off")
#     plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
#     plt.show()

    
# plot_slices(4, 4, size, resized[24:])